In [211]:
using Pkg
Pkg.activate(".")

In [212]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("bp_fourier.jl");

In [213]:
seed = 4
rng = MersenneTwister(seed)

T = 4
N = 3
A = [0 1 1; 
     1 0 0;
     1 0 0;]
# N = 4
# A = [0 1 0 0; 
#      1 0 1 0; 
#      0 1 0 1; 
#      0 0 1 0]
# N = 7
# A = [0 1 0 0 0 0 0; 
#      1 0 1 0 0 0 0; 
#      0 1 0 1 0 0 0;
#      0 0 1 0 1 0 0;
#      0 0 0 1 0 1 0;
#      0 0 0 0 1 0 1;
#      0 0 0 0 0 1 0]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
# N = 5
# A = [0 1 1 1 1; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]
# N = 8
# A = [0 1 1 1 0 0 1 0;
#      1 0 0 0 1 0 0 1; 
#      1 0 0 0 0 1 0 0; 
#      1 0 0 0 0 0 0 0; 
#      0 1 0 0 0 0 0 0; 
#      0 0 1 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0;
#      0 1 0 0 0 0 0 0]

g = IndexedBiDiGraph(A)

# N = 20
# c = 3
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedBiDiGraph(gg)
# A = zeros(Int,N,N)
# for i in CartesianIndices(A)
#      if !iszero(g.A[i])
#          A[i] = 1
#      end
# end

β = 1.0
J = 1
h = 0.0000001
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(4)

A

3×3 Matrix{Int64}:
 0  1  1
 1  0  0
 1  0  0

In [214]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
# ϕᵢ = [ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
# bp_fourier = deepcopy(bp)
iterate!(bp, maxiter=1; svd_trunc)

msg = deepcopy(bp.μ)
# jldsave("messages.jld2"; msg)

bp_fourier = deepcopy(bp)
# iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

In [215]:
node = 2
time = 4

ed = inedges(bp.g, node) |> first

display(ed)
display(msg[idx(ed)][time])

Indexed Edge 1 => 2 with index 1

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.16324   -0.130923
 -0.137504  -0.110281

[:, :, 2, 1] =
 -0.145459  -0.0927498
  0.172685   0.11011

[:, :, 1, 2] =
 -0.364177  0.0500324
 -0.30676   0.0421442

[:, :, 2, 2] =
 -0.307221   0.0529261
  0.364724  -0.0628323

In [216]:
iterate!(bp,maxiter=1; svd_trunc)

(1, CB_BP{ProgressUnknown, MatrixProductBP.var"#153#156"}(ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(25) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (9, 0, -1)), 0, 1, false, false, 1.737036256860245e9, 1.737036256860245e9, 1.737036256860245e9)), [[[2.0, 1.7199756388862464, 1.7457671703669857, 1.4534750798586558, 1.7221935652317724], [2.0, 1.9084414562055316, 1.73638324200243, 1.795063051371898, 1.5699882525094293], [2.0, 1.8681587644337014, 1.6795515511605423, 1.7001658312289776, 1.5797768916001604]], [[2.0, 1.9820137935704494, 1.8670989191164111, 1.8538934864441041, 1.769523241746139], [2.0, 1.8807970989765983, 1

In [217]:
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/50; svd_trunc)
;

In [218]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [219]:
er_b = []
ptr = []
for i in vertices(bp.g)
    margs = marginals(bp.b[i])
    margs_fourier = marginals(bp_fourier.b[i])
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er_b, abs(marg_fourier[ind]-marg[ind]))
        push!(ptr, (i, t))
    end
end

display(maximum(er_b))
display(ptr[argmax(er_b)])

0.39442158767598196

(2, 4)

In [220]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        push!(er_μ, abs(marg_fourier[1]-marg[1]))
        push!(ptr, (e, t))
    end
end

display(maximum(er_μ))
display(ptr[argmax(er_μ)])

0.5243995599399209

(Indexed Edge 1 => 2 with index 1, 3)

In [221]:
time = 3

display(bp.μ[1][time])
display(bp_fourier.μ[1][time])

4×4×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.096324    0.0587388  -0.0537664   0.032787
 -0.259268    0.158103   -0.144719    0.0882503
 -0.0189494  -0.135524   -0.0105772  -0.0756472
 -0.0510046  -0.36478    -0.0284699  -0.203614

[:, :, 2, 1] =
 -0.294432    0.179546    0.0383829   -0.0234061
  0.109388   -0.0667054  -0.0142601    0.00869588
 -0.0579223  -0.414254    0.00755089   0.0540032
  0.0215194   0.153905   -0.00280533  -0.0200634

[:, :, 1, 2] =
 -0.13031    -0.0417753  -0.0727367   -0.0233183
 -0.350745   -0.112443   -0.19578     -0.062764
 -0.0110883  -0.112275   -0.00618933  -0.0626698
 -0.0298457  -0.302201   -0.0166593   -0.168684

[:, :, 2, 2] =
 -0.398316   -0.127694    0.0519255    0.0166465
  0.147983    0.0474412  -0.0192915   -0.00618455
 -0.0338936  -0.343189    0.00441845   0.0447389
  0.0125922   0.127502   -0.00164156  -0.0166215

4×4×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.367715    -0.326517     0.208801   -0.0786567
 -0.0543813   -0.0627863   -0.129723    0.0550024
  0.00575243   0.00386086  -0.0170811   0.00696227
 -0.00303367   0.00500129   0.0869663  -0.0360171

[:, :, 2, 1] =
 -0.0569919    0.190847     0.209278    0.00141525
  0.0777989   -0.187456    -0.262487   -0.0730897
  0.0673684    0.111405    -0.140398   -0.329866
  0.00359664  -0.00638997  -0.0114122  -0.00559558

[:, :, 1, 2] =
 -0.407739     0.18795     -0.239555   -0.113754
 -0.0607272    0.0187834   -0.20081     0.0494377
  0.00634184  -0.00371546  -0.0104783   0.00747914
 -0.00313737   0.00633419   0.0858046  -0.036108

[:, :, 2, 2] =
 -0.0573204    0.220486     0.0721574     0.0424577
  0.0783142   -0.233954    -0.0473647    -0.137479
  0.0680654    0.0485154    0.150556     -0.416953
  0.00362255  -0.00872772  -0.000596716  -0.00883283

In [222]:
node = 2
time = 3

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

@show marg
@show marg_fourier;

In [223]:
node = 1
time = 3

edd = outedges(bp.g, node)
for ed in edd
    display(ed)
    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    marg = marginals(μ_marg)[time]

    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)
    marg_fourier = marginals(μ_fourier_marg)[time]

    display(marg)
    display(marg_fourier)
end

Indexed Edge 1 => 2 with index 1

2-element Vector{Float64}:
 0.31645052032057924
 0.6835494796794207

2-element Vector{Float64}:
 0.8408500802605002
 0.15914991973949977

Indexed Edge 1 => 3 with index 2

2-element Vector{Float64}:
 0.31645052032057924
 0.6835494796794207

2-element Vector{Float64}:
 0.8408500802605002
 0.15914991973949977

In [224]:
for ed in edges(bp.g)
    display(ed)

    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)

    for time in 1:T+1
        marg = marginals(μ_marg)[time]
        marg_fourier = marginals(μ_fourier_marg)[time]
        display("time = $time")
        display(marg)
        display(marg_fourier)
    end
end

Indexed Edge 1 => 2 with index 1

"time = 1"

2-element Vector{Float64}:
 -6.968122470822484e-17
  1.0

2-element Vector{Float64}:
 -1.6907445477422756e-16
  1.0000000000000002

"time = 2"

...

2-element Vector{Float64}:
 0.49999997900128285
 0.5000000209987171

2-element Vector{Float64}:
 0.4999999789957655
 0.5000000210042345

"time = 5"

2-element Vector{Float64}:
 0.4999999790012828
 0.5000000209987172

2-element Vector{Float64}:
 0.4999999789957658
 0.5000000210042341